# Carregamento de Dados

Objetivos dessa aula:
* Carregar um dataset customizado
* Implementar o fluxo de treinamento **e validação** completo de uma rede


## Hiperparâmetros

Vamos manter a organização do último script :)

* imports de pacotes
* configuração de hiperparâmetros
* definição do hardware padrão utilizado

E bora de GPU de novo! 


In [1]:
import torch
from torch import nn, optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

args = {
    'batch_size': 20,
    'num_workers': 4,
    'num_classes': 10,
    'lr': 1e-4,
    'weight_decay': 5e-4,
    'num_epochs': 30
}

if torch.cuda.is_available():
  args['device'] = torch.device('cuda')
else:
  args['device'] = torch.device('gpu')

print(args['device'])

cuda


## Dataset 

Dataset de aplicativos para aluguel de bicicletas (*Bike Sharing Dataset*). <br>
* Dadas algumas informações como velocidade do vento, estação do ano, etc., quantas bicicletas serão alugadas na próxima hora?

Esse é um problema de **Regressão**, onde precisamos estimar uma variável dependente em um espaço contínuo (alugueis de bikes) a partir de um conjunto de variáveis independentes (as condições no momento).

### Baixando o dataset

Fonte: https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset



In [2]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
! unzip Bike-Sharing-Dataset.zip

--2019-12-22 16:10:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279992 (273K) [application/x-httpd-php]
Saving to: ‘Bike-Sharing-Dataset.zip’

Bike-Sharing-Datase 100%[===================>] 273.43K   465KB/s    in 0.6s    

2019-12-22 16:10:35 (465 KB/s) - ‘Bike-Sharing-Dataset.zip’ saved [279992/279992]

Archive:  Bike-Sharing-Dataset.zip
  inflating: Readme.txt              
  inflating: day.csv                 
  inflating: hour.csv                


In [3]:
!ls

Bike-Sharing-Dataset.zip  day.csv  hour.csv  Readme.txt  sample_data


### Visualizando os dados

In [4]:
df = pd.read_csv('hour.csv')
print(len(df))
df.head()

17379


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Tratamento de dados

**Separação em treino e teste**<br>

Para treinar e validar o nosso modelo, precisamos de dois conjuntos de dados (treino e teste). Para isso, utilizaremos a função ```torch.randperm``` para amostrar aleatoriamente um percentual dos dados, separando-os para validação.

Documentação: https://pytorch.org/docs/stable/torch.html#torch.randperm

In [5]:
torch.manual_seed(1)
indices = torch.randperm(len(df)).tolist()

train_size = int(0.8*len(df))
df_train   = df.iloc[indices[:train_size]]
df_test    = df.iloc[indices[train_size:]]

print(len(df_train), len(df_test))
display(df_test.head())

df_train.to_csv('bike_train.csv', index=False)
df_test.to_csv('bike_test.csv', index=False)

!ls

13903 3476


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
12663,12664,2012-06-16,2,1,6,20,0,6,0,2,0.66,0.6212,0.47,0.1940,123,229,352
1801,1802,2011-03-20,1,0,3,18,0,0,0,1,0.38,0.3939,0.40,0.3582,58,98,156
16567,16568,2012-11-28,4,1,11,1,0,3,1,2,0.26,0.2576,0.75,0.2239,0,12,12
8817,8818,2012-01-08,1,1,1,5,0,0,0,2,0.32,0.3333,0.49,0.1045,0,2,2
2608,2609,2011-04-23,2,0,4,14,0,6,0,1,0.58,0.5455,0.78,0.3582,182,209,391


Bike-Sharing-Dataset.zip  bike_train.csv  hour.csv    sample_data
bike_test.csv		  day.csv	  Readme.txt


### Classe Dataset

O pacote ```torch.util.data``` possui a classe abstrata ```Dataset```. Ela permite que você implemente o seu próprio dataset reescrevendo os métodos:

* ```__init__(self)```: Define a lista de amostras do seu dataset
* ```__getitem__(self, idx)```: Carrega uma amostra, aplica as devidas transformações e retorna uma **tupla ```(dado, rótulo)```**.
* ```__len__(self)```: Retorna a quantidade de amostras do dataset

Tutorial completo do PyTorch: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html


In [0]:
class Bicicletinha(Dataset):
  def __init__(self, csv_path):
    self.dados = pd.read_csv(csv_path).to_numpy()

  def __getitem__(self, idx):
    
    sample = self.dados[idx][2:14]
    label  = self.dados[idx][-1:]

    # converte pra tensor
    sample = torch.from_numpy(sample.astype(np.float32))
    label  = torch.from_numpy(label.astype(np.float32))

    return sample, label

  def __len__(self):
    return len(self.dados)

### Construindo conjuntos de treino e teste

In [7]:
train_set = Bicicletinha('bike_train.csv')
test_set = Bicicletinha('bike_test.csv')

dado, rotulo = train_set[0]
print(rotulo)
print(dado)

tensor([373.])
tensor([ 4.0000,  1.0000, 11.0000, 19.0000,  0.0000,  4.0000,  1.0000,  1.0000,
         0.3800,  0.3939,  0.2700,  0.3582])


## Dataloader


In [0]:
train_loader = DataLoader(train_set, 
                          batch_size=args['batch_size'], 
                          shuffle=True, 
                          num_workers=args['num_workers'])

test_loader = DataLoader(test_set, 
                          batch_size=args['batch_size'], 
                          shuffle=True, 
                          num_workers=args['num_workers'])

O objeto retornado é um **iterador**, podendo ser utilizado para iterar em loops mas não suportando indexação.

In [9]:
for batch in train_loader:
  
  dado, rotulo = batch
  print(dado.size(), rotulo.size())
  break

torch.Size([20, 12]) torch.Size([20, 1])


## Implementando o MLP

Essa parte aqui você já tira de letra! Minha sugestão é construir um modelo com:

* **Duas camadas escondidas**. Lembre-se de alternar as camadas com ativações não-lineares. 
* Uma camada de saída (com qual ativação?)

In [0]:
class MLP(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(MLP, self).__init__()

    self.features  = nn.Sequential(
                      nn.Linear(input_size, hidden_size),
                      nn.ReLU(),
                      nn.Linear(hidden_size, hidden_size),
                      nn.ReLU()
                    )
    self.out     = nn.Linear(hidden_size, out_size)

  def forward(self, X):
    
    feature = self.features(X)
    output  = self.out(feature)

    return output

input_size  = len(train_set[0][0])
hidden_size = 128
out_size    = 1 # variaveis que serão preditas

net = MLP(input_size, hidden_size, out_size).to(args['device']) #cast na GPU 

## Definindo loss e otimizador

Se lembra quais as funções de perda adequadas para um problema de regressão?

In [0]:
criterion = nn.L1Loss().to(args['device'])
optimizer = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

# Fluxo de Treinamento & Validação

## Treinamento

Relembrando o passo a passo do fluxo de treinamento:
* Iterar nas épocas
* Iterar nos batches
* Cast dos dados no dispositivo de hardware
* Forward na rede e cálculo da loss
* Cálculo do gradiente e atualização dos pesos

Esse conjunto de passos é responsável pelo processo iterativo de otimização de uma rede. **A validação** por outro lado, é apenas a aplicação da rede em dados nunca antes visto para estimar a qualidade do modelo no mundo real.

## Validação

Para essa etapa, o PyTorch oferece dois artifícios:
* ```model.eval()```: Impacta no *forward* da rede, informando as camadas caso seu comportamento mude entre fluxos (ex: dropout).
* ```with torch.no_grad()```: Gerenciador de contexto que desabilita o cálculo e armazenamento de gradientes (economia de tempo e memória). Todo o código de validação deve ser executado dentro desse contexto.

Exemplo de código para validação

```python
net.eval()
with torch.no_grad():
  for batch in test_loader:
      # Código de validação
```

Existe o equivalente ao ```model.eval()``` para explicitar que a sua rede deve estar em modo de treino, é o ```model.train()```. Apesar de ser o padrão dos modelos, é boa prática definir também o modo de treinamento.

In [0]:
def forward(loader, net, epoch, mode):
  if mode == "train":
    net.train()
  else:
    net.eval()

  epoch_loss = []
  for batch in loader:
    dado, rotulo = batch

    # Cast na GPU
    dado   = dado.to(args['device'])
    rotulo = rotulo.to(args['device'])

    # Forward 
    pred = net(dado)
    loss = criterion(pred, rotulo)
    epoch_loss.append(loss.cpu().data)

    if mode == "train":
      # Backward
      loss.backward()
      optimizer.step()

  epoch_loss = np.asarray(epoch_loss)
  print("Epoca %d, Loss: %.4f +\- %.4f" % (epoch, epoch_loss.mean(), epoch_loss.std()) )

In [0]:
def train(train_loader, net, epoch):
  net.train()
  
  epoch_loss = []
  for batch in train_loader:
    
    dado, rotulo = batch

    # Cast na GPU
    dado   = dado.to(args['device'])
    rotulo = rotulo.to(args['device'])

    # Forward 
    pred = net(dado)
    loss = criterion(pred, rotulo)
    epoch_loss.append(loss.cpu().data)

    # Backward
    loss.backward()
    optimizer.step()

  epoch_loss = np.asarray(epoch_loss)

  print("Epoca %d, Loss: %.4f +\- %.4f" % (epoch, epoch_loss.mean(), epoch_loss.std()) )


In [0]:
def test(test_loader, net, epoch):

  net.eval()
  with torch.no_grad():
    epoch_loss = []
    for batch in test_loader:
      
      dado, rotulo = batch

      # Cast na GPU
      dado   = dado.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Forward 
      pred = net(dado)
      loss = criterion(pred, rotulo)
      epoch_loss.append(loss.cpu().data)

    epoch_loss = np.asarray(epoch_loss)

    print("Epoca %d, Loss: %.4f +\- %.4f" % (epoch, epoch_loss.mean(), epoch_loss.std()) )


In [19]:
for epoch in range(args['num_epochs']):
  forward(train_loader, net, epoch, "train")
  forward(test_loader, net, epoch, "test")
  print("-------------------------------")

Epoca 0, Loss: 62.8757 +\- 17.5248
Epoca 0, Loss: 59.4253 +\- 15.5799
-------------------------------
Epoca 1, Loss: 62.8531 +\- 17.8940
Epoca 1, Loss: 60.5900 +\- 17.8724
-------------------------------
Epoca 2, Loss: 61.5132 +\- 18.5312


KeyboardInterrupt: ignored

In [15]:
for epoch in range(args['num_epochs']):
  train(train_loader, net, epoch)
  test(test_loader, net, epoch)
  print("-------------------------------")

Epoca 0, Loss: 72.2474 +\- 19.4387
Epoca 0, Loss: 70.3291 +\- 19.5188
-------------------------------
Epoca 1, Loss: 71.1552 +\- 19.6303
Epoca 1, Loss: 71.3996 +\- 17.6912
-------------------------------
Epoca 2, Loss: 71.3632 +\- 19.8101
Epoca 2, Loss: 67.6779 +\- 18.0414
-------------------------------
Epoca 3, Loss: 70.4939 +\- 20.0094
Epoca 3, Loss: 68.9646 +\- 19.6207
-------------------------------
Epoca 4, Loss: 68.9333 +\- 19.5214
Epoca 4, Loss: 67.2910 +\- 16.9556
-------------------------------
Epoca 5, Loss: 68.6479 +\- 19.2051
Epoca 5, Loss: 66.8890 +\- 20.6597
-------------------------------
Epoca 6, Loss: 68.1833 +\- 18.4450
Epoca 6, Loss: 65.0332 +\- 18.0561
-------------------------------
Epoca 7, Loss: 68.1598 +\- 18.8672
Epoca 7, Loss: 67.9391 +\- 17.8428
-------------------------------
Epoca 8, Loss: 66.8093 +\- 18.2120
Epoca 8, Loss: 64.6980 +\- 18.6455
-------------------------------
Epoca 9, Loss: 65.8938 +\- 18.8063
Epoca 9, Loss: 62.9662 +\- 16.5044
------------

KeyboardInterrupt: ignored

In [16]:
Xtest = torch.stack([tup[0] for tup in test_set])
Xtest = Xtest.to(args['device'])

ytest = torch.stack([tup[1] for tup in test_set])
ypred = net(Xtest).cpu().data

data = torch.cat((ytest, ypred), axis=1)

df_results = pd.DataFrame(data, columns=['ypred', 'ytest'])
df_results.head(20)

,ypred,ytest
0,tensor(352.),tensor(328.6181)
1,tensor(156.),tensor(117.4714)
2,tensor(12.),tensor(-6.2506)
3,tensor(2.),tensor(-18.1174)
4,tensor(391.),tensor(419.7492)
5,tensor(391.),tensor(355.4768)
6,tensor(84.),tensor(200.9507)
7,tensor(487.),tensor(547.9028)
8,tensor(176.),tensor(194.9202)
9,tensor(157.),tensor(98.5586)


# Gráfico de convergência

In [17]:
plt.figure(figsize=(20, 9))
plt.plot(train_losses, label='Train')
plt.plot(test_losses, label='Test', linewidth=3, alpha=0.5)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Convergence', fontsize=16)
plt.legend()
plt.show()

NameError: ignored

<Figure size 1440x648 with 0 Axes>